# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
%tensorflow_version 2.x


In [0]:
import tensorflow as tf
import numpy as np

In [0]:
(X_train, y_train), (X_test, y_test) =tf.keras.datasets.cifar10.load_data()


In [5]:
 X_train.shape

(50000, 32, 32, 3)

In [24]:
X_test.shape

(10000, 32, 32, 3)

In [0]:
#Transform x_train and x_test on scale of 0-1
x_train_scaled = np.subtract(X_train, np.min(X_train))/np.subtract(np.max(X_train),np.min(X_train))
x_test_scaled = np.subtract(X_test, np.min(X_test))/np.subtract(np.max(X_test),np.min(X_test))


In [0]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

In [0]:
n_samples = 50000

In [0]:
for ix in range(n_samples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train_scaled[ix])
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train_scaled[ix])
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test_scaled[ix])
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test_scaled[ix])
        Y2_test.append(y_test[ix])

In [0]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

In [12]:
X1_train.shape

(25000, 32, 32, 3)

In [13]:
X2_train.shape

(25000, 32, 32, 3)

In [14]:
X1_test.shape

(5000, 32, 32, 3)

In [15]:
X2_test.shape

(5000, 32, 32, 3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
Y1_train = tf.keras.utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = tf.keras.utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = tf.keras.utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = tf.keras.utils.to_categorical(np.asarray(Y2_test), num_classes=10)


In [26]:
Y1_train.shape

(25000, 5)

In [27]:
Y1_test.shape

(5000, 5)

In [28]:
Y2_train.shape

(25000, 10)

In [29]:
Y2_test.shape

(5000, 10)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
#Importing the Libraries for building ANN model 
from tensorflow.keras.models import Sequential  #For building the Neural Network layer by layer
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Reshape, Flatten #Dense - To randomly initialize the weights to small numbers close to 0(But not 0)
from tensorflow.keras.layers import MaxPool2D, Conv2D
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.backend import backend

In [0]:
tf.keras.backend.clear_session()
#Initialize model, reshape & normalize data
model = Sequential()

model.add(Reshape((32,32,3),input_shape=(32,32,3)))

#normalize data
model.add(BatchNormalization())

In [0]:
#Add convolutional layer
model.add(Conv2D(64, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))


#Add convolutional layer
model.add(Conv2D(64, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))




In [0]:
#Add MaxPooling layer
model.add(MaxPool2D(pool_size=(2,2)))


#Add another dropout layer
model.add(Dropout(0.42))

In [0]:
#Flatten the output
model.add(Flatten())

In [0]:
#normalize data
model.add(BatchNormalization())
#Fully Connected Dense layer - 1st
model.add(Dense(100, activation='relu'))

#normalize data
model.add(BatchNormalization())
#Dense layer - 2nd
model.add(Dense(100, activation='relu'))

#normalize data
model.add(BatchNormalization())
#Dense layer - 3rd
model.add(Dense(10, activation='relu'))

In [0]:
model.add(Dense(5, activation='softmax'))

In [0]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [142]:
#Train the model
model.fit(X1_train,Y1_train,          
          validation_data=(X1_test,Y1_test),
          epochs=10,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 166s 7ms/sample - loss: 0.9797 - accuracy: 0.6078 - val_loss: 0.7793 - val_accuracy: 0.6934
Epoch 2/10
25000/25000 [==============================] - 164s 7ms/sample - loss: 0.7627 - accuracy: 0.7055 - val_loss: 0.6987 - val_accuracy: 0.7314
Epoch 3/10
25000/25000 [==============================] - 165s 7ms/sample - loss: 0.6477 - accuracy: 0.7524 - val_loss: 0.6763 - val_accuracy: 0.7424
Epoch 4/10
25000/25000 [==============================] - 165s 7ms/sample - loss: 0.5802 - accuracy: 0.7828 - val_loss: 0.6497 - val_accuracy: 0.7612
Epoch 5/10
25000/25000 [==============================] - 165s 7ms/sample - loss: 0.5314 - accuracy: 0.8004 - val_loss: 0.6183 - val_accuracy: 0.7724
Epoch 6/10
25000/25000 [==============================] - 164s 7ms/sample - loss: 0.4797 - accuracy: 0.8191 - val_loss: 0.6467 - val_accuracy: 0.7630
Epoch 7/10
25000/25000 [===========================

In [0]:
#Able to achieve accuracy of 78.30% on test data.

In [143]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [144]:
print(len(model.layers))

14


In [0]:
#Set conv layers to be non-trainable
for layer in model.layers[:7]:
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
transfer_model = Sequential(model.layers[:7])


In [0]:
#Fully Connected Dense layer - 1st
transfer_model.add(Dense(200, activation='relu'))

#normalize data
transfer_model.add(BatchNormalization())
#Dense layer - 2nd
transfer_model.add(Dense(200, activation='relu'))

#normalize data
transfer_model.add(BatchNormalization())
#Dense layer - 3rd
transfer_model.add(Dense(100, activation='relu'))

In [0]:
transfer_model.add(Dense(10, activation='softmax'))

In [0]:
transfer_model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [154]:
transfer_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)            

In [153]:
#Train the model
transfer_model.fit(X2_train,Y2_train,          
          validation_data=(X2_test,Y2_test),
          epochs=10,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 65s 3ms/sample - loss: 0.6272 - accuracy: 0.7730 - val_loss: 0.4472 - val_accuracy: 0.8346
Epoch 2/10
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.4152 - accuracy: 0.8486 - val_loss: 0.4384 - val_accuracy: 0.8446
Epoch 3/10
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.3377 - accuracy: 0.8756 - val_loss: 0.4124 - val_accuracy: 0.8518
Epoch 4/10
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.2823 - accuracy: 0.8964 - val_loss: 0.3928 - val_accuracy: 0.8640
Epoch 5/10
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.2387 - accuracy: 0.9110 - val_loss: 0.4020 - val_accuracy: 0.8602
Epoch 6/10
25000/25000 [==============================] - 62s 2ms/sample - loss: 0.2039 - accuracy: 0.9269 - val_loss: 0.4276 - val_accuracy: 0.8570
Epoch 7/10
25000/25000 [==============================] -

In [0]:
#Able to achieve accuracy of 86.82% on test data.

# Text classification using TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [66]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [68]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [69]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [70]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [80]:
twenty_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [81]:
twenty_test.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [82]:
twenty_test.data[0:5]

["From: brian@ucsd.edu (Brian Kantor)\nSubject: Re: HELP for Kidney Stones ..............\nOrganization: The Avant-Garde of the Now, Ltd.\nLines: 12\nNNTP-Posting-Host: ucsd.edu\n\nAs I recall from my bout with kidney stones, there isn't any\nmedication that can do anything about them except relieve the pain.\n\nEither they pass, or they have to be broken up with sound, or they have\nto be extracted surgically.\n\nWhen I was in, the X-ray tech happened to mention that she'd had kidney\nstones and children, and the childbirth hurt less.\n\nDemerol worked, although I nearly got arrested on my way home when I barfed\nall over the police car parked just outside the ER.\n\t- Brian\n",
 'From: rind@enterprise.bih.harvard.edu (David Rind)\nSubject: Re: Candida(yeast) Bloom, Fact or Fiction\nOrganization: Beth Israel Hospital, Harvard Medical School, Boston Mass., USA\nLines: 37\nNNTP-Posting-Host: enterprise.bih.harvard.edu\n\nIn article <1993Apr26.103242.1@vms.ocom.okstate.edu>\n banschbach@

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [173]:
# create a document-term matrix using TF-IDF
#Fit transform Training data
vect = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=0.20, ngram_range=(1,2))
dtm = vect.fit_transform(twenty_train.data)
features_train = vect.get_feature_names()
dtm.shape

(2257, 31568)

In [180]:
dtm

<2257x31568 sparse matrix of type '<class 'numpy.float64'>'
	with 312336 stored elements in Compressed Sparse Row format>

In [181]:
features_train

['00',
 '00 00',
 '00 08',
 '00 09',
 '00 34',
 '00 57',
 '00 58',
 '00 gmt',
 '00 p003228',
 '00 pm',
 '00 thank',
 '000',
 '000 000',
 '000 bytes',
 '000 colors',
 '000 people',
 '000 pounds',
 '000 question',
 '000 year',
 '0001',
 '0001 south',
 '0010580b',
 '01',
 '01 02',
 '01 08',
 '01 09',
 '01 10',
 '01 11',
 '01 19',
 '01 55',
 '01 differences',
 '01 lines',
 '01 v2',
 '0100',
 '01580',
 '01580 change',
 '02',
 '02 1993',
 '02 42',
 '02 51',
 '02 55',
 '0200',
 '02118',
 '02173',
 '02173 617',
 '023044',
 '023044 19580',
 '024103',
 '024103 29880',
 '03',
 '03 01',
 '03 02',
 '03 03',
 '03 07',
 '03 11',
 '03 1993',
 '03 24',
 '03 25',
 '03 26',
 '03 44',
 '03 gmt',
 '0349',
 '0349 athens',
 '0358',
 '0358 athens',
 '0358 university',
 '04',
 '04 12',
 '04 13',
 '04 17',
 '04 1993',
 '04 gmt',
 '040',
 '040 stacks',
 '041343',
 '041343 24997',
 '05',
 '05 01',
 '05 10',
 '05 11',
 '05 13',
 '05 14',
 '05 30',
 '05 31',
 '05 32',
 '05 33',
 '0511',
 '06',
 '06 1993',
 '0600',


In [174]:
#Transform testing data
dtm1 = vect.transform(twenty_test.data)
features_test = vect.get_feature_names()
dtm1.shape

(1502, 31568)

In [179]:
dtm1

<1502x31568 sparse matrix of type '<class 'numpy.float64'>'
	with 176284 stored elements in Compressed Sparse Row format>

In [178]:
features_test

['00',
 '00 00',
 '00 08',
 '00 09',
 '00 34',
 '00 57',
 '00 58',
 '00 gmt',
 '00 p003228',
 '00 pm',
 '00 thank',
 '000',
 '000 000',
 '000 bytes',
 '000 colors',
 '000 people',
 '000 pounds',
 '000 question',
 '000 year',
 '0001',
 '0001 south',
 '0010580b',
 '01',
 '01 02',
 '01 08',
 '01 09',
 '01 10',
 '01 11',
 '01 19',
 '01 55',
 '01 differences',
 '01 lines',
 '01 v2',
 '0100',
 '01580',
 '01580 change',
 '02',
 '02 1993',
 '02 42',
 '02 51',
 '02 55',
 '0200',
 '02118',
 '02173',
 '02173 617',
 '023044',
 '023044 19580',
 '024103',
 '024103 29880',
 '03',
 '03 01',
 '03 02',
 '03 03',
 '03 07',
 '03 11',
 '03 1993',
 '03 24',
 '03 25',
 '03 26',
 '03 44',
 '03 gmt',
 '0349',
 '0349 athens',
 '0358',
 '0358 athens',
 '0358 university',
 '04',
 '04 12',
 '04 13',
 '04 17',
 '04 1993',
 '04 gmt',
 '040',
 '040 stacks',
 '041343',
 '041343 24997',
 '05',
 '05 01',
 '05 10',
 '05 11',
 '05 13',
 '05 14',
 '05 30',
 '05 31',
 '05 32',
 '05 33',
 '0511',
 '06',
 '06 1993',
 '0600',


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [184]:
# use logistic regression with all features - Training accuracy
logreg = LogisticRegression(C=1e9)
logreg.fit(dtm, twenty_train.target)
y_pred_class = logreg.predict(dtm)
print("Training Accuracy : ",metrics.accuracy_score(twenty_train.target, y_pred_class))

Training Accuracy :  1.0


In [185]:
#Testing Accuracy
y_pred_class_test = logreg.predict(dtm1)
y_pred_class_test

array([2, 2, 2, ..., 2, 2, 1])

In [186]:
print("Testing Accuracy : ",metrics.accuracy_score(twenty_test.target, y_pred_class_test))

Testing Accuracy :  0.9247669773635153
